# Article Collections with Opensearch

_OpenSearch Python Client Documentation: https://opensearch-project.github.io/opensearch-py/_

In [7]:
import opensearchpy
from opensearchpy import OpenSearch
import json
from chameleon import PageTemplate
from IPython.core.display import HTML


In [3]:
# Connecting to OpenSearch
# https://opensearch.org/docs/latest/clients/python-low-level/#connecting-to-opensearch

host = 'localhost'
port = 9200
auth = ('admin', 'admin') # For testing only. Don't store credentials in code.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = False, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [4]:
# Searching for documents; client.search()
# https://opensearch.org/docs/latest/clients/python-low-level/#searching-for-documents

q = 'lorem'
# Create a query that matches documents containing the term 'lorem' 
# in the 'standard_html' field and 'ZMSFolder' in the field meta_id
query = {
  'size': 5,
  'query': {
    'bool': {
      'must': [
        {
          'multi_match': {
            'query': q,
            'fields': ['standard_html']
          }
        },
        {
          'term': {
            'meta_id': 'ZMSFolder'
          }
        }
      ]
    }
  }
}


# Use the query dictionary directly
response = client.search( index = 'zmsdemo', body = json.dumps(query) )

# Print hits
if 'hits' in response and 'hits' in response['hits']:
    # Print the total number of hits
    print(f"Total hits: {response['hits']['total']['value']}")
    print(10* '-')
    hits = response['hits']['hits']
    for hit in hits:
        print(f"ID: {hit['_id']}, Score: {hit['_score']}")
        # print(f"Source: {json.dumps(hit['_source'], indent=2)}")
        print(f"Title: {json.dumps(hit['_source']['title'], indent=2)}")
        print(f"Descr: {json.dumps(hit['_source']['attr_dc_description'], indent=2)}")
        print(f"meta_id: {json.dumps(hit['_source']['meta_id'], indent=2)}")
        print(10* '-')
else:
    print("No hits found.")

Total hits: 7
----------
ID: uid:783386fc-bb6c-47bf-b298-440eec09976b:ger, Score: 2.458997
Title: "ZMS Download"
Descr: "Dogubergren, no sea takimata sanctus est Lorem"
meta_id: "ZMSFolder"
----------
ID: uid:fbb75a7e-af64-4ee1-9bce-6c314f150dfc:ger, Score: 2.4589012
Title: "Professional Services"
Descr: "The professional services of the ZMS development team support professional users from companies and institutions in the maximal using of the advantages of the ZMS framework."
meta_id: "ZMSFolder"
----------
ID: uid:423bbfe6-b859-4357-b559-71231ddd485c:ger, Score: 2.4582415
Title: "ZMS Download"
Descr: "Dogubergren, no sea takimata sanctus est Lorem"
meta_id: "ZMSFolder"
----------
ID: uid:db773d67-93e8-4def-9efc-74c4098023f0:ger, Score: 2.4581165
Title: "ZMS Academy"
Descr: "Behind ZMS stands a conceptional approach for an efficient and low cost production of websites or web documents. You get to know this approach in detail in the courses of the ZMS academy."
meta_id: "ZMSFolder"
---

In [26]:
# Define the template as a string with Chameleon syntax for repeating hits
pt = PageTemplate("""
<main>
    <article class="infoxbox" tal:repeat="hit hits">
        <div tal:define="title hit['_source']['title']; attr_dc_description hit['_source']['attr_dc_description']">
            <h1 tal:content="title">Title</h1>
            <p tal:content="attr_dc_description">Description</p>
        </div>
    </article>
</main>
""")
# Prepare the CSS styles for the HTML output
css = """
    <style>
        main {
            display: flex;
            flex-direction: row;
            padding: 20px;
            font-family: Arial, sans-serif;
            color: #333;
            column-gap: 40px;
            flex-wrap: wrap;
            justify-content: flex-start;
        }
        .infoxbox {
            background-color: #f0f0f0;
            border: 1px solid #ccc;
            padding: 10px;
            margin: 10px 0;
            width: 250px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            border-radius: 5px;
            transition: transform 0.2s;
        }
        .infoxbox h1 {
            color: #333;
            font-size: 1.25em;
        }
        .infoxbox p {
            color: #666;
        }
    </style>
"""

html_code = [css]  # Start with the CSS styles
if hits:
    html_code.append(pt.render(hits=hits))
# Display the rendered HTML in Jupyter Notebook
display(HTML(''.join(html_code)))